In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix, accuracy_score,classification_report
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import tree

In [ ]:
#Getting data, path may change after we are done with google colab
cases_train = pd.read_excel('../data/cases_2021_train_processed_2.xlsx')
cases_test = pd.read_excel('../data/cases_2021_test_processed_unlabelled_2.xlsx')

In [ ]:
#Altering some feartures in cases_train
#combine province and country as one feature
cases_train["province"].fillna("",inplace=True)
cases_train["country_province"] = cases_train["country"] +"_"+cases_train["province"]

#Instead of raw numbers on active and recovered, changed them to ratio like Death
cases_train["Case_Recovery_Ratio"] = cases_train["Recovered"] / cases_train["Confirmed"]
cases_train["Case_Active_Ratio"] = cases_train["Active"] / cases_train["Confirmed"]

#Change date and chronic disease datatype to object for mapping and balancing
cases_train["date_confirmation"] = cases_train["date_confirmation"].dt.strftime('%Y-%m-%d')
cases_train["chronic_disease_binary"] = cases_train["chronic_disease_binary"].map({True: 'True',False: 'False'})

#Dropping features that are no longer needed 
drop_col = ["province", "country","latitude","longitude","Deaths","Recovered","Active"]
cases_train.drop(drop_col,axis=1,inplace=True)

#=====>Should we alter datetime to an obj or a float since it is ordinal? It is currently an obj<=====

In [ ]:
#Altering some feartures in cases_test
#combine province and country as one feature
cases_test["province"].fillna("",inplace=True)
cases_test["country_province"] = cases_test["country"] +"_"+cases_test["province"]

#Instead of raw numbers on active and recovered, changed them to ratio like Death
cases_test["Case_Recovery_Ratio"] = cases_test["Recovered"] / cases_test["Confirmed"]
cases_test["Case_Active_Ratio"] = cases_test["Active"] / cases_test["Confirmed"]

#Change date and chronic disease datatype to object for mapping and balancing
cases_test["date_confirmation"] = cases_test["date_confirmation"].dt.strftime('%Y-%m-%d')
cases_test["chronic_disease_binary"] = cases_test["chronic_disease_binary"].map({True: 'True',False: 'False'})

#Dropping features that are no longer needed 
drop_col = ["province", "country","latitude","longitude","Deaths","Recovered","Active"]
cases_test.drop(drop_col,axis=1,inplace=True)



In [ ]:
#cases_train.groupby(['outcome_group']).size()

In [ ]:
#Mapping outcome_group to numerical values as instructed in 1.2
cases_train.outcome_group = pd.Categorical(cases_train.outcome_group)
cases_train["outcome_group"] = cases_train.outcome_group.cat.codes

In [ ]:
#cases_test

In [ ]:
#Splitting the feature we need to predict in the test case
X = cases_train.drop("outcome_group",axis = 1)
y = cases_train.outcome_group 

In [ ]:
#Balance the dataset via oversampling using SMOTENC for non majority categorical classes
oversample = SMOTENC(categorical_features=[X.dtypes==object],random_state = 101,sampling_strategy="not majority")
X_processed, y_processed = oversample.fit_resample(X,y)

In [ ]:
#Mapping all categorical values to numerical via dummy encoding since most of them are not ordinal
X_processed = pd.get_dummies(X_processed, columns=X_processed.select_dtypes(exclude="number").columns)
cases_test = pd.get_dummies(cases_test, columns=cases_test.select_dtypes(exclude="number").columns)

In [ ]:
#Prevent feature mismatch
X_processed, test_processed = X_processed.align(cases_test, join='inner', axis=1) 

In [ ]:
#Split the traning data to a set of new train data and test data at a 80:20 ratio 

X_train, X_test, y_train, y_test = train_test_split(X_processed, y_processed,test_size=0.20)

In [ ]:
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse is {"{:.2f}".format(rmse)}')

In [ ]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 20)

max_depth = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

for depth in max_depth:
    score = cross_val_score(tree.DecisionTreeRegressor(max_depth = depth, random_state = 20), X_processed, y_processed, cv = kf, scoring = "neg_mean_squared_error")
    print(f'Max Depth: {depth}')
    rmse(score.mean())

min_split = [2,3,4,5,6,7,8,9,10]

for split in min_split:
    score = cross_val_score(tree.DecisionTreeRegressor(min_samples_split = split, random_state = 20), X_processed, y_processed, cv = kf, scoring = "neg_mean_squared_error")
    print(f'Min samples split: {split}')
    rmse(score.mean())

min_samples_leaf = [1,2,3,4,5,6,7,8,9,10]

for leaf in min_split:
    score = cross_val_score(tree.DecisionTreeRegressor(min_samples_leaf = leaf, random_state = 20), X_processed, y_processed, cv = kf, scoring = "neg_mean_squared_error")
    print(f'min samples leaf: {leaf}')
    rmse(score.mean())
  

In [ ]:
decisionTree = tree.DecisionTreeClassifier(criterion = 'gini', splitter='best', max_depth = 20, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)

decisionTree.fit(X_train, y_train)

y_prediction = decisionTree.predict(X_test)

numpy_y_test = y_test.to_numpy()

y_true = numpy_y_test

In [ ]:
macroF1Score = f1_score(y_true, y_prediction, average='macro')
print("Macro F1 score: ", macroF1Score)

averageAccuracy = decisionTree.score(X_test, y_test)
print("Average Accuracy: ", averageAccuracy)

weightedF1Score = f1_score(y_true, y_prediction, average='weighted')
print("Mean F1 score on Deceased: ", weightedF1Score)

In [ ]:
#Setting up Kfold and xgb classifier for hyper tuning
kf = KFold(n_splits=5,random_state = 10, shuffle = True)
xgb_cl = xgb.XGBClassifier()

In [ ]:
#Hypertuning xgb_cl and getting the F1 score and accuracy
#Since this cell takes a very long time since we use the GridSearchCV
"""
param_grid = {
    "max_depth": [3,4,5,7],
    "min_child_weight": [1,3,5],
    "gamma": [0,0.25,1],
    "learning_rate": [0.1,0.15,0.05],
    "subsample":[0.8],
    "tree_method":["hist"],
    "colsample_bytree": [0.5],
}

scorings = {"F1_Macro": "f1_macro","Acc": "accuracy"}

xgb_cl = xgb.XGBClassifier(use_label_encoder=False)

grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=kf, scoring=scorings,refit="F1_Macro")

grid_cv.fit(X_train, y_train)

#grid_cv.best_score_

best_param = grid_cv.best_params_

print(best_param)

param_table = pd.DataFrame(grid_cv.cv_results_)
param_table = param_table[["params","mean_test_F1_Macro","mean_test_Acc","rank_test_F1_Macro"]]
best_val = param_table.loc[param_table["rank_test_F1_Macro"] == 1]
param_table.to_csv('results/xgboost_tuning.txt', sep='\t', index=False)

print(best_val)

"""

In [ ]:
#set up tuned xgb classifier for the classification report from the validatoin set
xgb_cl_t = xgb.XGBClassifier(gamma = 0.25,max_depth = 7,min_child_weight = 1,learning_rate = 0.15,colsample_bytree=0.5,tree_method='hist')

xgb_cl_t.fit(X_train,y_train)

y_pred = xgb_cl_t.predict(X_test)

print(classification_report(y_test, y_pred))
print("Macro F1:",f1_score(y_test, y_pred,average='macro'))

In [ ]:
#plotting confusion matrix
cf_matrix = confusion_matrix(y_test,y_pred)

categories = ["Deceased","Hospitalized","Non-Hospitalized"]
h_map = sns.heatmap(cf_matrix/np.sum(cf_matrix),fmt='.2%', annot=True,xticklabels= categories,yticklabels = categories)
h_map.set_xlabel('True label', fontsize=12)
h_map.set_ylabel('Predicted Label', fontsize=12)
h_map.set_title('Confusion Matrix of Validation Set For XGBoost', fontsize=15)
plt.savefig('plots/plot 1.png')

In [ ]:
#set up data for overfitting graph
depth_val = [i for i in range(1, 15)]
train_scores, test_scores = list(), list()

In [ ]:
#create data for overfitting graph
"""
for i in depth_val:

    xgb_cl_t = xgb.XGBClassifier(gamma = 0.25,max_depth = i,min_child_weight = 1,learning_rate = 0.15,colsample_bytree=0.5,tree_method='hist')

    xgb_cl_t.fit(X_train, y_train)

    train_pred = xgb_cl_t.predict(X_train)
    train_acc = accuracy_score(y_train, train_pred)
    train_scores.append(train_acc)

    test_pred = xgb_cl_t.predict(X_test)
    test_acc = accuracy_score(y_test, test_pred)
    test_scores.append(test_acc)
    

    print('>%d, train: %.3f, test: %.3f' % (i, train_acc, test_acc))

"""

In [ ]:
#plot overfitting graph
"""
plt.plot(depth_val, train_scores, '-o', label='Train')
plt.plot(depth_val, test_scores, '-o', label='Test')
plt.legend()
plt.title("XGBoost accuracy on Train/Test data for different max_depth") 
plt.xlabel("max_depth value") 
plt.ylabel("Accuracy Score") 
plt.savefig('plots/plot 2.png')
"""

In [ ]:
#Use final model to predict cases_test

model = xgb.XGBClassifier(gamma = 0.25,max_depth = 7,min_child_weight = 1,learning_rate = 0.15,colsample_bytree=0.5,tree_method='hist')
model.fit(X_processed,y_processed)

prediction = model.predict(test_processed)

In [ ]:
#Converting prediction to csv
pred_df = pd.DataFrame(data = prediction,columns = ["Prediction"])
pred_df["Id"] = pred_df.index

pred_df = pred_df[["Id","Prediction"]]
pred_df.to_csv('predictions.csv', index=False)